# SCRIPT FOR ABNORMAL VITAL SIGNS PREDICTION

In [1]:
!pip install tensorflowjs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.6 MB/s 
     |████████████████████████████████| 40 kB 4.2 MB/s 
     |████████████████████████████████| 180 kB 32.0 MB/s 
     |████████████████████████████████| 145 kB 47.5 MB/s 
     |████████████████████████████████| 217 kB 49.6 MB/s 
     |████████████████████████████████| 51 kB 6.7 MB/s 
     |████████████████████████████████| 85 kB 4.2 MB/s 
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3


In [2]:
# Read the csv taking as separator the comma and create the dataframe
vital_data = pd.read_csv("Human_vital_signs_R.csv", sep=',')
vital_data.head()

,Unnamed: 0,Time (s),HR (BPM),RESP (BPM),SpO2 (%),TEMP (*C),OUTPUT
0,0,0,94.0,21.0,97.0,36.2,Normal
1,1,1,94.0,25.0,97.0,36.2,Normal
2,2,2,101.0,25.0,93.0,38.0,Abnormal
3,3,3,55.0,11.0,100.0,35.0,Abnormal
4,4,4,93.0,26.0,95.0,37.0,Normal


We want to know more about our data. The values are int64 and float64, and the rows are 25,000.

In [3]:
vital_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25493 entries, 0 to 25492
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   25493 non-null  int64  
 1   Time (s)     25493 non-null  int64  
 2    HR (BPM)    25488 non-null  float64
 3    RESP (BPM)  25346 non-null  float64
 4    SpO2 (%)    25366 non-null  float64
 5   TEMP (*C)    25493 non-null  float64
 6   OUTPUT       25493 non-null  object 
dtypes: float64(4), int64(2), object(1)
memory usage: 1.4+ MB


The function describe() does some useful computations.

In [4]:
vital_data.describe()

,Unnamed: 0,Time (s),HR (BPM),RESP (BPM),SpO2 (%),TEMP (*C)
count,25493.000000,25493.000000,25488.000000,25346.000000,25366.000000,25493.000000
mean,240.000000,239.981132,89.127943,17.640496,96.716471,37.590123
std,138.855163,138.855230,13.220448,3.589381,3.323381,5.211265
min,0.000000,-1.000000,44.000000,0.000000,83.000000,21.000000
25%,120.000000,120.000000,81.000000,16.000000,95.000000,34.000000
50%,240.000000,240.000000,89.000000,18.000000,97.000000,38.000000
75%,360.000000,360.000000,95.000000,20.000000,99.000000,41.000000
max,480.000000,480.000000,139.000000,34.000000,111.000000,49.000000


We need to clean the data, taking out 0 values.

In [5]:
# First lets check how many 0 values are in these columns 
featureList = [' HR (BPM)', ' RESP (BPM)', ' SpO2 (%)', 'TEMP (*C)']
vital_data[featureList].isin([0]).sum()

 HR (BPM)        0
 RESP (BPM)    219
 SpO2 (%)        0
TEMP (*C)        0
dtype: int64

Let's replace zeros with the mean.
Define the function in charge of cleaning the data. NaN and infinite not allowed. Moreover, all values need to be in float32 for sklearn.

In [6]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [7]:
# The first two columns are not needed for our scope
vital_data = vital_data.drop("Unnamed: 0", axis=1)
vital_data = vital_data.drop("Time (s)", axis=1)

# For Heart Rate
vital_data[" HR (BPM)"] = vital_data[" HR (BPM)"].replace({ 0 : vital_data[" HR (BPM)"].mean()})
vital_data[" HR (BPM)"] = vital_data[" HR (BPM)"].astype(np.float32)

# For Respiration Rate
vital_data[" RESP (BPM)"] = vital_data[" RESP (BPM)"].replace({ 0 : vital_data[" RESP (BPM)"].mean()})
vital_data[" RESP (BPM)"] = vital_data[" RESP (BPM)"].astype(np.float32)

# For Oxygen Saturation
vital_data[" SpO2 (%)"] = vital_data[" SpO2 (%)"].replace({ 0 : vital_data[" SpO2 (%)"].mean()})
vital_data[" SpO2 (%)"] = vital_data[" SpO2 (%)"].astype(np.float32)

# For Body Temperature
vital_data["TEMP (*C)"] = vital_data["TEMP (*C)"].replace({ 0 : vital_data["TEMP (*C)"].mean()})
vital_data["TEMP (*C)"] = vital_data["TEMP (*C)"].astype(np.float32)

# Substitute normal with 0 and abnormal with 1
vital_data = vital_data.replace({'Normal': 0, 'Abnormal': 1})
vital_data["OUTPUT"] = vital_data["OUTPUT"].astype(np.float32)

# Lets checkif it worked
vital_data[featureList].isin([0]).sum()
print(vital_data.dtypes)
print(vital_data['TEMP (*C)'].head)

print(vital_data)
vital_data = vital_data.reset_index(drop=True)
vital_data = clean_dataset(vital_data)

 HR (BPM)      float32
 RESP (BPM)    float32
 SpO2 (%)      float32
TEMP (*C)      float32
OUTPUT         float32
dtype: object
<bound method NDFrame.head of 0        36.200001
1        36.200001
2        38.000000
3        35.000000
4        37.000000
           ...    
25488    33.000000
25489    36.400002
25490    36.200001
25491    37.000000
25492    37.000000
Name: TEMP (*C), Length: 25493, dtype: float32>
        HR (BPM)   RESP (BPM)   SpO2 (%)  TEMP (*C)  OUTPUT
0           94.0         21.0       97.0  36.200001     0.0
1           94.0         25.0       97.0  36.200001     0.0
2          101.0         25.0       93.0  38.000000     1.0
3           55.0         11.0      100.0  35.000000     1.0
4           93.0         26.0       95.0  37.000000     0.0
...          ...          ...        ...        ...     ...
25488       56.0         12.0      101.0  33.000000     1.0
25489       94.0         25.0       98.0  36.400002     0.0
25490       94.0         21.0       97.0  36

Now we do the train/test split.

In [8]:
print(vital_data.shape)
X = vital_data.drop(["OUTPUT"], axis=1)
y = vital_data["OUTPUT"]

print(X.shape)
print(y.shape)

(25214, 5)
(25214, 4)
(25214,)


Without other parameters, 75% of the dataset is prepared for training and 25% for test.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
# train set 75% + test set 25%

In [10]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

We can find out which model has the best score without considering the parameters yet.

In [11]:
from sklearn.model_selection import KFold, cross_val_score
for model in [ 
    DummyClassifier,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    GaussianNB,
    SVC,
    RandomForestClassifier]:
    
    cls = model()
    kf = KFold(n_splits = 5)
    score = cross_val_score(cls, X_train, y_train, cv = kf, scoring="roc_auc", error_score='raise')
    
    print(f"{model.__name__:22}  AUC: \t {score.mean():.3f} STD: {score.std():.2f}")

DummyClassifier         AUC: 	 0.500 STD: 0.00
DecisionTreeClassifier  AUC: 	 0.996 STD: 0.00
KNeighborsClassifier    AUC: 	 0.999 STD: 0.00
GaussianNB              AUC: 	 0.994 STD: 0.00
SVC                     AUC: 	 0.995 STD: 0.00
RandomForestClassifier  AUC: 	 1.000 STD: 0.00


It turns out the Random Forest is the best choice.

In [12]:
# Fitting the modle model
cls = RandomForestClassifier()

# Fitting the model
cls.fit(X_train, y_train)

# Prediction
y_preds = cls.predict(X_test)

In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1421
         1.0       1.00      1.00      1.00      4883

    accuracy                           1.00      6304
   macro avg       1.00      1.00      1.00      6304
weighted avg       1.00      1.00      1.00      6304



In [14]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_preds))

0.9961611202490134


## FINE TUNING

In [15]:
from sklearn.model_selection import RandomizedSearchCV

# Define a grid of hyperparameters
grid = {"n_estimators": [10, 100, 200, 500, 1000, 1200],
        "max_depth": [None, 5, 10, 20, 30],
        "max_features": ["auto", "sqrt"],
        "min_samples_split": [2, 4, 6],
        "min_samples_leaf": [1, 2, 4]}

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Set n_jobs to -1 to use all cores (NOTE: n_jobs=-1 is broken as of 8 Dec 2019, using n_jobs=1 works)
clf = RandomForestClassifier(n_jobs=1)

# Setup RandomizedSearchCV
rs_clf = RandomizedSearchCV(estimator=clf,
                            param_distributions=grid,
                            n_iter=10, # try 10 models total
                            cv=5, # 5-fold cross-validation
                            verbose=2) # print out results

# Fit the RandomizedSearchCV version of clf
rs_clf.fit(X_train, y_train);

# Find the best hyperparameters
print(rs_clf.best_params_)

# Scoring automatically uses the best hyperparameters
rs_clf.score(X_test, y_test)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   8.9s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   8.9s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   8.9s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   8.7s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200; total time=   8.9s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   8.8s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   8.8s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimat

0.9977791878172588

In [16]:
print(f"Final Score: {rs_clf.score(X_test, y_test)}")

Final Score: 0.9977791878172588


## SAVE THE MODEL

In [17]:
import joblib
import pickle

In [18]:
filename = 'finalized_model.sav'
pickle.dump(rs_clf, open(filename, 'wb'))

In [19]:
print(X_test)
print(X_test.shape)
model = pickle.load(open(filename, 'rb'))
# Last check
p = model.predict(X_test)
print(p)

        HR (BPM)   RESP (BPM)   SpO2 (%)  TEMP (*C)
20192       84.0         18.0       95.0       42.0
25153      100.0         20.0      100.0       37.0
19966       84.0         17.0       95.0       31.0
23306       85.0         20.0       86.0       41.0
21095       88.0         16.0       97.0       41.0
...          ...          ...        ...        ...
24641      115.0         19.0       95.0       36.0
2931        90.0         20.0       95.0       27.0
17130       74.0         19.0       98.0       40.0
22279       84.0         22.0       98.0       40.0
717         92.0         15.0      100.0       29.0

[6304 rows x 4 columns]
(6304, 4)
[1. 1. 0. ... 1. 1. 0.]
